### We create the initial array at first, for both versions of the algorithm.

In [2]:
import pandas as pd
import numpy as np

# Creating the database with the ratings
user_ratings = pd.DataFrame({'Movie 1':[5.0, 4.0, 3.0, 2.0], 'Movie 2': [3, 2, None, 5],
                             'Movie 3':[None, 1, None, 1], 'Movie 4': [1, None, 1, 5],
                             'Movie 5': [None, None, 3, 3], 'Movie 6': [None, 1, 3, 4]},
                             index = ['User X', 'User Y', 'User Z', 'User W'])
user_ratings

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6
User X,5.0,3.0,NaN,1.0,NaN,NaN
User Y,4.0,2.0,1.0,NaN,NaN,1.0
User Z,3.0,NaN,NaN,1.0,3.0,3.0
User W,2.0,5.0,1.0,5.0,3.0,4.0


### Implementation of the first version of algorithm.
In the first version, we will calculate the cosine similarity between user X and the other users, and we will calculate the score for cell (X, 6) as the weighted average of the score of the two most similar users in X.

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import heapq

A =  np.array([user_ratings.loc['User X'].values,
               user_ratings.loc['User Y'].values,
               user_ratings.loc['User Z'].values,
               user_ratings.loc['User W'].values])

A_sparse = sparse.csr_matrix(A)
A_sparse = np.nan_to_num(A_sparse.toarray())

similarities = cosine_similarity(A_sparse)

# Print Similarity table for User X
from prettytable import PrettyTable
t = PrettyTable(['Similarities','User X', 'User Y', 'User Z', 'User W'])
t.add_row(['User X', similarities[0][0], similarities[0][1], similarities[0][2], similarities[0][3]])
print(t)

# Get the 2 largest values and its' index value
index = heapq.nlargest(2, range(1,len(similarities[0])), similarities[0].take)
values = [similarities[0][index[0]], similarities[0][index[1]]]
prediction = values[0]*user_ratings.iloc[index[0]].values[5] + values[1]*user_ratings.iloc[index[1]].values[5]
prediction /= values[0]+values[1]
print('\n'+'\033[1m'+'\033[4m'+'The prediction for the review (X,6) is:'+'\033[1m'+'\033[0m', prediction)

+--------------+--------+--------------------+--------------------+--------------------+
| Similarities | User X |       User Y       |       User Z       |       User W       |
+--------------+--------+--------------------+--------------------+--------------------+
|    User X    |  1.0   | 0.9369749612033813 | 0.5111012519999519 | 0.5669467095138409 |
+--------------+--------+--------------------+--------------------+--------------------+

The prediction for the review (X,6) is: 2.1309366449453377


### Implementation of the second version of algorithm.
We will first subtract the average value of the scores from each line (will use only non-zero values), then again calculate the cosine similarity of user X with other users, and now calculate the deviation from the mean for cell (X, 6) to the weighted average of the deviation of the two most similar users to X. Finally, we calculate the score by adding the deviation we calculated to the mean value of user X.

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import heapq

mean_ratings_dev = user_ratings.sub(user_ratings.mean(axis = 1, skipna = True), axis = 0)
print('\n'+'\033[1m'+'\033[4m'+'User ratings subtracting each rows\' mean value:'+'\033[1m'+'\033[0m')
print(mean_ratings_dev)

A =  np.array([mean_ratings_dev.loc['User X'].values,
               mean_ratings_dev.loc['User Y'].values,
               mean_ratings_dev.loc['User Z'].values,
               mean_ratings_dev.loc['User W'].values])

A_sparse = sparse.csr_matrix(A)
A_sparse = np.nan_to_num(A_sparse.toarray())

similarities = cosine_similarity(A_sparse)
# Print Similarity table for User X
from prettytable import PrettyTable
t = PrettyTable(['Similarities','User X', 'User Y', 'User Z', 'User W'])
t.add_row(['User X', similarities[0][0], similarities[0][1], similarities[0][2], similarities[0][3]])
print('\n'+'\033[1m'+'\033[4m'+'Similarity table for User X'+'\033[1m'+'\033[0m')
print(t)

index = heapq.nlargest(2, range(1,len(similarities[0])), similarities[0].take)
values = [similarities[0][index[0]], similarities[0][index[1]]]

prediction = values[0]*mean_ratings_dev.iloc[index[0]].values[5] + values[1]*mean_ratings_dev.iloc[index[1]].values[5]
prediction /= values[0]+values[1]
prediction += user_ratings.loc['User X'].mean(skipna = True)
print('\n'+'\033[1m'+'\033[4m'+'The prediction for the review (X,6) is:'+'\033[1m'+'\033[0m',prediction)


User ratings subtracting each rows' mean value:
         Movie 1   Movie 2   Movie 3   Movie 4   Movie 5   Movie 6
User X  2.000000  0.000000       NaN -2.000000       NaN       NaN
User Y  2.000000  0.000000 -1.000000       NaN       NaN -1.000000
User Z  0.500000       NaN       NaN -1.500000  0.500000  0.500000
User W -1.333333  1.666667 -2.333333  1.666667 -0.333333  0.666667

Similarity table for User X
+--------------+--------------------+--------------------+-------------------+---------------------+
| Similarities |       User X       |       User Y       |       User Z      |        User W       |
+--------------+--------------------+--------------------+-------------------+---------------------+
|    User X    | 0.9999999999999998 | 0.5773502691896258 | 0.816496580927726 | -0.5809475019311124 |
+--------------+--------------------+--------------------+-------------------+---------------------+

The prediction for the review (X,6) is: 2.878679656440357


<br><br><br>
## Results and algorithms analysis.


$\;\;\;$We notice that there is a significant difference in the two cases, this is because in the first version of the algorithm we calculate the estimate (Χ,6) based on user ratings, while in the second we get the deviation from the average value of ratings for each user.<br><br>
$\;\;\;$Thus, for each case the cosine similarities are different.
In the first version the users with the greatest similarity are Υ with cos_sim(X,Y)=0,93 and user W with cos_sim(X,W)=0.56, while in the second version are users Ζ and Υ with cos_sim(X,Z)=0.81 and cos(X,Y)=0.57 respectively.


$\;\;\;$These changes exist because each version 'works' differently. Specifically for the first version, the similarity is calculated based on the values of the scores, while in the second, the similarity between the standard deviation of the scores plays a role. 

$\;\;\;$In the first version the users with the greatest weight in terms of the result are Y=1 and W=4 with Υ having higher weight cos_sim(X,Y)=0.936 than user W with cos_sim(X,W)=0,56. In the case that we look user Y in relation to user X, is very similar because in the same movies they have given similar scores, for 'Movie 1' X and Y have scored 5 and 4 respectively as well as in movie 2 they have given 3 and 2 respectively, so the numerator in the formula of similarity increases significantly. However, this does not apply to user W in relation to user X, where they have rated the same movies with quite different ratings. User X gives 5 and 3 as we said for 'Movie 1' and 'Movie 2', while user W with 2 and 5 respectively. In movie 6, Y gives 1 and user W 4, so intuitively we could say that the score will be clearly closer to that given by user Y than the user W. And so it is with 2.13.<br>

$\;\;\;$In the second case we look at different users. The biggest similarity with user X, have the users Z with value 0.816 and user Y with 0.577. Here user Z influences user X more and gives a score of 0.5 (with the new table) for movie 6 with movies having the greatest impact are 'Movie 1', 'Movie 5' and 'Movie 6' because they give a very small standard deviation, being more consistent in his choices and thus there is greater similarity. User Y gives -1. Therefore the score should be closer to that of user Ζ and in combination with the negative rating given by Υ the predicted review is 2.87.